# <font color=blue><div align="center">Projet Karhoo</div></font>


In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

#Module json
import json

import time as tm


# Objectif1 : maximiser le nombre de courses possibles aux chauffeurs


<font color=blue><div align="center"></div></font>

$$\mbox{Maximize} \;\;\; \sum_{k \in K} \; \sum_{i \in P} \; \sum_{j \in N} \;x_{ij}^{k} $$

# Objectif 2: minimiser le temps total de trajet pour les chauffeurs


<font color=blue><div align="center"></div></font>

$$\mbox{Minimize} \;\;\; \sum_{k \in K} \; \sum_{i \in N} \; \sum_{j \in N} \;x_{ij}^{k} t_{ij}$$

# Traitement de la base de données

In [2]:
n = 25 
Nombres_de_shifts = 8
stations = N = range(52)
chauffeurs = K = range(1,9)
Pickups = P = range(1, 26)
Dropoffs = D = range(26, 51)

# Small_sized_data : dict
small_sized_data = "/Users/mac/Desktop/Supelec/OPTI des trans passagers/DATA projet Karhoo/day_data.json" #insérer le chemin du fichier


# Chargement des données: 
with open(small_sized_data) as SmallSizedData:
    reader = json.load(SmallSizedData)
    bookings = reader["bookings"]
    shifts = reader["shifts"]
    
  

In [3]:
Stations_demandes={} #dictionaire qui lie chaque noeud avec la station réelle 
id_stations={}
for i in range (n):
    Stations_demandes[i+1]=bookings[i]['jobs'][0]['station']
    id_stations[i+1]=bookings[i]['jobs'][0]["id"]
    Stations_demandes[n+i+1]=bookings[i]['jobs'][1]['station']
    id_stations[n+i+1]=bookings[i]['jobs'][1]["id"]
    bookings[i]['id']=i+1
    bookings[i]['jobs'][0]['station']=i+1
    bookings[i]['jobs'][1]['station']=n+i+1


In [4]:
Stations_demandes[0]='s0'
Stations_demandes[51]='s0'

for k in range (Nombres_de_shifts):
    shifts[k]['id']=k+1
    shifts[k]['jobs'][0]['station']=0
    shifts[k]['jobs'][1]['station']=51



In [5]:
#la durée maximale de chaque trajet
T={}
for k in chauffeurs:
    T[k]=shifts[k-1]['jobs'][1]["timeDate"]-shifts[k-1]['jobs'][0]["timeDate"] 

In [6]:
#dict des charges:
q=[0,]
for r in range(n):
    q.append(bookings[r]["passengers"])
for r in range(n):
    q.append(-bookings[r]["passengers"])
q.append(0)



# MLP

In [7]:
m = Model("PL-OBJ1")
travel_time = "/Users/mac/Desktop/Supelec/OPTI des trans passagers/DATA projet Karhoo/travel_times4.csv"  #insérer le chemin du fichier

Time = {}

number_of_s = 52

with open(travel_time) as TravelTime:
    reader = csv.DictReader(TravelTime)
    for row in reader:
        b = int(row['station'])
        for sId in range(0, number_of_s +1):
            Time[(b,sId)] = float(row[f's{sId}'])
            
time = {}
stations = range(52)
for i in stations:
    for j in stations:
        d = int(Stations_demandes[i][1:])
        a = int(Stations_demandes[j][1:])
        time[(i, j)] = Time[(d, a)]
    

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-27


## Définition des variables

| Nom Variable 	| Type (entier, binaire, continue) 	| Description 	|
|:------------:	|:--------------------------------:	|:-----------:	|
|  xijk    	|            binaire        	|  passage ou non du chauffeur k par l'arc (i,j) 	|
|  Bik 	|            entier            	|  le temps où le chauffeur k commence le service au noeud i	|
|  Lik 	|            entier           	| le temps de trajet du client i dans la véhicule k	|
|  Qik  	|            entier            	|  la charge de la véhicule k après sa visite du noeud i 	|

In [8]:

Xijk = {(i, j, k) : m.addVar(vtype = GRB.BINARY, name = f'x{i}_{j}_{k}') for i in stations for j in stations for k in chauffeurs}
Bik = {(i, k) : m.addVar(vtype = GRB.INTEGER, name = f'B{i}_{k}') for i in stations for k in chauffeurs}
Lik = {(i,k): m.addVar(vtype= GRB.INTEGER, name = f'L{i}_{k}') for i in Pickups for k in chauffeurs}
Qik=  {(i,k): m.addVar(vtype= GRB.INTEGER, name = f'Q{i}_{k}') for i in stations for k in chauffeurs}


## Définition des contraintes

<font color=blue><div align="center"></div></font>

$$\mbox{Chaque demande est servie une seule fois : } \;\;\; \sum_{k\in K }  \sum_{j \in N} x_{ij}^{k} <= 1 \;\;\;   pour  \;  tout  \; i\in P $$ 

In [9]:

#Contrainte 1 :

x_constr1 = {}
for i in P:
    x_constr1[i] = m.addConstr(quicksum([Xijk[(i,j,k)] for j in stations for k in chauffeurs]) <= 1, name = f'constr1{i}')


<font color=blue><div align="center"></div></font>

$$ \;\;\; \sum_{k\in K }  \sum_{i \in N} x_{ii}^{k} = 0 \;\;\;$$ 

In [10]:
x_constr67={}
for k in chauffeurs:
    x_constr67[k]=  m.addConstr(quicksum([Xijk[(i,i,k)] for i in stations]) == 0, name = f'constr5{k}')

<font color=blue><div align="center"></div></font>

$$\mbox{the above constraint not correct and should be: } \;\;\;  \sum_{i \in N} x_{i,2n+1}^{k} = 1 \;\;\; $$ 

In [11]:
x_constr51={}
for k in chauffeurs :
    x_constr51[k]= m.addConstr(quicksum([Xijk[(i,51,k)] for i in range(51)]) == 1, name = f'constr51{k}')


<font color=blue><div align="center"></div></font>

$$\mbox{Chaque demande est servie une seule fois : } \;\;\; \sum_{j\in N } x_{ij}^{k} - \sum_{j\in N } x_{n+i,j}^{k} = 0 \;\;\;   pour  \;  tout  \; i\in P \; pour  \;  tout  \; k\in K $$  

In [12]:
#contrainte2
x_constr2 = {}
for k in chauffeurs :
    for i in P:
        x_constr2[i,k] = m.addConstr(quicksum([Xijk[(i,j,k)] for j in stations]) - quicksum([Xijk[(n+i,j,k)] for j in stations]) == 0, name = f'constr2{i,k}')

<font color=blue><div align="center"></div></font>

$$\mbox{La voiture commence par la station de départ : } \;\;\; \sum_{j\in N } x_{0j}^{k} = 1 \;\;\;   pour  \;  tout  \; k\in K  $$ 

In [13]:
#Contrainte 3 : 

x_constr3 = {}
for k in chauffeurs :
    x_constr3[k] = m.addConstr(quicksum([Xijk[(0,j,k)] for j in range(1,52)]) == 1, name = f'constr3{k}')


<font color=blue><div align="center"></div></font>

$$\mbox{Chaque client pris de la station doit être déposé  } \;\;\; \sum_{j\in N } x_{ji}^{k} - \sum_{j\in N } x_{ij}^{k} = 0 \;\;\;   pour  \;  tout  \; i\in P\cup\ D \; pour  \;  tout  \; k\in K $$

In [14]:
#Contrainte 4 : 

x_constr4 = {}
for k in  chauffeurs:
    for i in range(1, 51):
        Xji=quicksum([Xijk[(j,i,k)] for j in stations])
        Xij=quicksum([Xijk[(i,j,k)] for j in stations])
                      
        x_constr4[i,k] = m.addConstr(Xji-Xij == 0, name = f'constr4{i,k}')

$$ e_{i} : Le \; temps \; le  \;plus  \; tôt  \; où  \;la  \; course  \; peut  \; commencer \; au \;noeud\;  i  si\; i \in P\cup\ D \; \;   $$ 
$$ l_{i} : Le \; temps \; le  \;plus  \; tard  \; où  \;la  \; course  \; peut  \; commencer \; au \;noeud\;  i  \; si \;i \in P\cup\ D \;  $$ 


In [15]:
#dict des time window 
e={}
l={}
for i in P:
    E=bookings[i-1]['jobs'][0]['timeWindowBeginDate']
    L=bookings[i-1]['jobs'][0]['timeWindowEndDate']
    e[i]=E
    l[i]=L
for i in P:
    E=bookings[i-1]['jobs'][1]['timeWindowBeginDate']
    L=bookings[i-1]['jobs'][1]['timeWindowEndDate']   
    e[n+i]=E
    l[n+i]=L



$$ e_{i} = min(min(e_{i} - t[0,i] \; for\; i \in \; P), min(e_{k})) $$
$$ l_{i} = max(max(l_{i} - t[i,51] \; for\; i \in \; D), max(l_{k})) $$

In [16]:
e[0]=min(min(e[i]-time[0,i] for i in Pickups),min(shifts[k-1]["jobs"][0]["timeDate"] for k in K))
e[51]=min(min(e[i]-time[0,i] for i in Pickups),min(shifts[k-1]["jobs"][0]["timeDate"] for k in K))
l[0]=max(max(l[i]+time[i,51] for i in Dropoffs),max(shifts[k-1]["jobs"][1]["timeDate"] for k in K))
l[51]=max(max(l[i]+time[i,51] for i in Dropoffs),max(shifts[k-1]["jobs"][1]["timeDate"] for k in K))


$$ M_{ij}^{k} \; tel\; que: \; max \{0,l_{i}+d_{i}+t_{ij}-e_{i}\} = M_{ij}^{k} $$ 


In [17]:
#definition de Mijk
M={}

for k in chauffeurs:
    for i in N:   
        for j in N:
            M[(i,j,k)]=max(0,l[i]+60+time[(i,j)]-e[j])
      

<font color=blue><div align="center"></div></font>

$$\mbox{La cohérence de la variable temps  }  (B_{i}^{k} + d_{i}+ t_{ij}) x_{ij}^{k} \leq  B_{j}^{k}  \;\;\;   pour  \;  tout  \; i\in N \; , pour  \;  tout  \; j\in N \; , pour  \;  tout  \; k\in K $$


<font color=blue><div align="center"></div></font>

$$\mbox{contrainte 6 linéarisée }  B_{i}^{k} + d_{i}+ t_{ij}-M_{ij}^{k}(1-x_{ij}^{k}) \leq  B_{j}^{k}  \;\;\;   pour  \;  tout  \; i\in N \; , pour  \;  tout  \; j\in N \; , pour  \;  tout  \; k\in K $$

In [18]:
#Contrainte6
B_constr6={}
for k in chauffeurs:
    for i in N:
        for j in N:
            if i==0 or i==51:
                d=0
            else :
                d=60
            B_constr6[(i,j,k)]= m.addConstr(Bik[(j,k)]  >= (Bik[(i,k)] + d + time[(i,j)] -  M[(i,j,k)] * (1 - Xijk[(i,j,k)])) , name = f'constr6{i}_{j}_{k}')


$$ W_{ij}^{k} \; tel\; que: \; min \{Q_{k},+Q_{k}+q_{i}\} = W_{ij}^{k} $$ 

In [19]:
#definition de Wijk
W={}
for k in chauffeurs : 
    Q=shifts[k-1]["capacity"]
    for j in N:
        for i in N:
            W[(i,j,k)]=min(Q,Q+q[i])


<font color=blue><div align="center"></div></font>

$$\mbox{La cohérence de la variable charge  }  (Q_{i}^{k} + q_{j}) x_{ij}^{k} \leq  Q_{j}^{k}  \;\;\;   pour  \;  tout  \; i\in N \; , pour  \;  tout  \; j\in N \; , pour  \;  tout  \; k\in K $$

<font color=blue><div align="center"></div></font>

$$\mbox{contrainte 7 linéarisée }  Q_{i}^{k} + q_{i}-W_{ij}^{k}(1-x_{ij}^{k}) \leq  Q_{j}^{k}  \;\;\;   pour  \;  tout  \; i\in N \; , pour  \;  tout  \; j\in N \; , pour  \;  tout  \; k\in K $$

In [20]:
#contrainte7
Q_constr={(i,j,k):m.addConstr(Qik[(j,k)]>=Qik[(i,k)]+q[j]-W[(i,j,k)]*(1-Xijk[(i,j,k)]), name='f_Constr7_{i}_{j}_{k}') for i in N for j in N for k in chauffeurs}       


<font color=blue><div align="center"></div></font>

$$\mbox{Le temps de trajet de chaque client : }  L_{i}^{k} = B_{n+i}^{k} - (B_{i}^{k} + d_{i}) pour  \;  tout  \; i\in P \;  , pour  \;  tout  \; k\in K $$

In [21]:
#contrainte8
L_constr_1={(i,k): m.addConstr(Lik[(i,k)]== Bik[(n+i,k)]-Bik[(i,k)]-60, name=f'L_constr{(i,k)}') for k in chauffeurs for i in Pickups}            
    
    

<font color=blue><div align="center"></div></font>

$$\mbox{Contrainte pour borner la durée de chaque route:  }  (B_{2n+1}^{k} - B_{0}^{k} ) \leq  T^{k}  \;\;\;   pour  \;  tout  \; k\in K $$

In [22]:
#contrainte9
B_constr_1 ={}
B_constr_2 = {}
for k in chauffeurs:
    B_constr_1[k] = m.addConstr(Bik[(51,k)] <= shifts[k-1]['jobs'][1]["timeDate"], name=f'B_constr_1{k}')
    B_constr_2[k] = m.addConstr(Bik[(0,k)] >= shifts[k-1]['jobs'][0]["timeDate"], name=f'B_constr_2{k}')

<font color=blue><div align="center"></div></font>

$$\mbox{Contrainte sur l’heure de début de la course  }  e_{i} \leq B_{i}^{k}  \leq  l_{i}  \;\;\;   pour  \;  tout  \; i\in N \; pour  \;  tout  \; k\in K $$

In [23]:
#contrainte 10
B_constr_10_inf={(i,k):m.addConstr(e[i]<=Bik[(i,k)], name= f'Constr_11_inf_{i,k}') for i in N for k in chauffeurs}
B_constr_10_sup={(i,k):m.addConstr(l[i]>=Bik[(i,k)], name= f'Constr_11_sup_{i,k}') for i in N for k in chauffeurs}


In [24]:
#Contrainte chauffeurs
B_constr_chauffeur={m.addConstr(shifts[k-1]['jobs'][0]["timeDate"] <=Bik[(0,k)], name= f'Constr_chauffeur_{0,k}') for k in chauffeurs}
B_constr_chauffeur={m.addConstr(shifts[k-1]['jobs'][1]["timeDate"] <=Bik[(51,k)], name= f'Constr_chauffeur_{51,k}') for k in chauffeurs}

<font color=blue><div align="center"></div></font>

$$\mbox{Contrainte de durée maximale que le client ne doit pas dépasser : }  t_{i,n+i} \leq L_{i}^{k}  \leq  L  \;\;\;   pour  \;  tout  \; i\in P \; pour  \;  tout  \; k\in K $$

In [25]:
#contrainte 11
L_constr_3={(i,k): m.addConstr(Lik[(i+1,k)]<= bookings[i]['maximumDuration'], name=f'L_constr_3{(i+1,k)}') for i in range(0,25) for k in chauffeurs}
for k in chauffeurs:
    for i in range(1,26): 
        L_constr_4={(i,k): m.addConstr(Lik[(i,k)]>= time[i,n+i], name=f'L_constr_4{(i,k)}')}


$$\mbox{Contrainte de capacité de véhicule k: }  max (0,q_{i}) \leq Q_{i}^{k}  \leq  min(Q_{k},Q_{k}+ q_{i})  \;\;\;   pour  \;  tout \; \; i\in N \; pour  \;  tout  \; k\in K $$

In [26]:
#Contrainte 12 
Q_constrr={}
Q_constr2={}
for k in chauffeurs:
    Q=shifts[k-1]["capacity"]
    for i in N:
        Q_constrr[(i,k)] = m.addConstr(Qik[(i,k)] >= max(0,q[i]) , name = f'Q_constr{(i,k)}')
        Q_constr2[(i,k)] = m.addConstr(Qik[(i,k)] <= min(Q,Q+q[i]) , name = f'Q_constr{(i,k)}')

<font color=blue><div align="center"></div></font>

$$\mbox{Contrainte sur le revenu maximal de chaque chauffeur } \;\;\; \sum_{j\in N }  \sum_{i \in P} x_{ij}^{k} r_{i} \leq R_{k}  \;\;\;   pour  \;  tout  \; k\in K \;  $$ 

In [27]:
#contrainte salaire
for k in chauffeurs:
    P_Constr={k: m.addConstr(quicksum([bookings[i-1]["price"]*Xijk[(i,j,k)] for i in Pickups for j in stations])<=shifts[k-1]["maximumTurnover"])}


# Objectif1:

In [28]:
# -- Ajout de la fonction objectif --

        
z=quicksum([Xijk[(i,j,k)] for i in Pickups for j in stations for k in chauffeurs ])
m.setObjective(z, GRB.MAXIMIZE)

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update()

#-- Affichage en mode texte du PL --
m.display()

In [29]:
start_time = tm.time()
# -- Résolution --
m.optimize()

In [30]:
from operator import itemgetter
if m.status == GRB.INFEASIBLE:
    print("\n LE PROGRAMME N'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print("\n LE PROGRAMME EST NON BORNÉ!!!")
else:
    client={}
    res=0
    L=[]
    attrib={}
    chemin={}
    for k in chauffeurs:
        chauffeur=[]
        chemin[k]=[]
        for i in N:
            for j in N:
                if Xijk[(i,j,k)].x==1 :
                    chauffeur.append((i,j,Bik[(i,k)].x))
                    res += time[(i,j)]
                    if i not in chemin[k]:
                        chemin[k].append(i)
                    if j not in chemin[k]:
                        chemin[k].append(j)
            attrib[k]=chauffeur
        c=0
        for i in Pickups:
            for j in N:
                if Xijk[(i,j,k)].x==1 :
                    c+=1
        client[k]=c
        print("Le chauffeur {} effectue les courses suivantes: {}".format(k,sorted(chauffeur, key = itemgetter(2))))
        L.append(sorted(chauffeur, key = itemgetter(2)))
        print("Le chauffeur {} a  {} clients ".format(k,client[k]))

print("Nombre maximal des clients servis :{}".format(sum(client[k] for k in chauffeurs)))
print("Temps total de trajet :{}".format(res))



print('Le temps d\'exécution de cette première instance est: {} secondes'.format(tm.time() - start_time))

Le chauffeur 1 effectue les courses suivantes: [(0, 19, 45000.0), (19, 44, 46092.0), (44, 18, 47210.0), (18, 15, 48802.0), (15, 43, 49098.0), (43, 40, 49734.0), (40, 13, 50429.0), (13, 38, 52500.0), (38, 51, 53131.0)]
Le chauffeur 1 a  4 clients 
Le chauffeur 2 effectue les courses suivantes: [(0, 6, 31836.0), (6, 31, 37177.0), (31, 51, 37731.0)]
Le chauffeur 2 a  1 clients 
Le chauffeur 3 effectue les courses suivantes: [(0, 9, 52708.0), (9, 34, 53700.0), (34, 17, 54664.0), (17, 42, 55500.0), (42, 24, 56173.0), (24, 49, 62093.0), (49, 51, 63089.0)]
Le chauffeur 3 a  3 clients 
Le chauffeur 4 effectue les courses suivantes: [(0, 25, 36600.0), (25, 1, 42540.0), (1, 26, 42600.0), (26, 3, 43941.0), (3, 50, 44158.0), (50, 20, 44409.0), (20, 45, 44821.0), (45, 28, 45346.0), (28, 12, 45406.0), (12, 37, 46800.0), (37, 51, 47394.0)]
Le chauffeur 4 a  5 clients 
Le chauffeur 5 effectue les courses suivantes: [(0, 2, 48600.0), (2, 27, 49800.0), (27, 11, 50584.0), (11, 36, 53700.0), (36, 10, 5450

In [31]:
chemin={}
for k in range(len(L)):
    chemin[k+1] = []
    for i in range(len(L[k])):
        chemin[k+1].append(L[k][i][0])
    chemin[k+1].append(51)

In [32]:
chemin

{1: [0, 19, 44, 18, 15, 43, 40, 13, 38, 51],
 2: [0, 6, 31, 51],
 3: [0, 9, 34, 17, 42, 24, 49, 51],
 4: [0, 25, 1, 26, 3, 50, 20, 45, 28, 12, 37, 51],
 5: [0, 2, 27, 11, 36, 10, 35, 5, 30, 51],
 6: [0, 23, 48, 7, 32, 51],
 7: [0, 16, 41, 8, 4, 29, 33, 14, 39, 51],
 8: [0, 22, 47, 21, 46, 51]}

In [31]:
c=0
for v in client.values():
    c+=v    #c nombre de clients servies

In [32]:
with open(small_sized_data) as SmallSizedData:
    reader = json.load(SmallSizedData)
    booking = reader["bookings"]
    shift = reader["shifts"]
dict={"nb_assigned_bookings":c, 'route_cost': res, 'shifts': []}
for k in chauffeurs:
    dict1={'id':shift[k-1]['id'],'jobs':[]}
    for i in chemin[k]:
        if i==0:
            dict1['jobs'].append({'id' : shift[k-1]["jobs"][0]['id'], 'time':Bik[(0,k)].x})
           
        if i!=0 and i!=51:
            dict_pu={}
            dict_do={}
            if i<=25:
                dict_pu['id']=id_stations[i]
                dict_pu['time']= Bik[(i,k)].x
                dict_do['id']=id_stations[25+i]
                dict_do['time']= Bik[(25+i,k)].x  
                dict1['jobs'].append(dict_pu)
                dict1['jobs'].append(dict_do)
        if i==51:
            dict1['jobs'].append({'id' : shift[k-1]["jobs"][1]['id'], 'time':Bik[(51,k)].x})
            
    dict['shifts'].append(dict1)

    print (dict1)
    
out_file = open("/Users/mac/Desktop/sortie.json","w")
json.dump(dict, out_file)
out_file.close()


{'id': 3238042, 'jobs': [{'id': -9714126, 'time': 45000.0}, {'id': 24601133, 'time': 46092.0}, {'id': 24601134, 'time': 47210.0}, {'id': 25132978, 'time': 52500.0}, {'id': 25132979, 'time': 53131.0}, {'id': 25060099, 'time': 49098.0}, {'id': 25060100, 'time': 50429.0}, {'id': 24243975, 'time': 48802.0}, {'id': 24243977, 'time': 49734.0}, {'id': -9714125, 'time': 58800.0}]}
{'id': 3238029, 'jobs': [{'id': -9714087, 'time': 31836.0}, {'id': 24102822, 'time': 37177.0}, {'id': 24102823, 'time': 37731.0}, {'id': -9714086, 'time': 41400.0}]}
{'id': 3237957, 'jobs': [{'id': -9713871, 'time': 52708.0}, {'id': 25136432, 'time': 53700.0}, {'id': 25136433, 'time': 54664.0}, {'id': 24244025, 'time': 55500.0}, {'id': 24244027, 'time': 56173.0}, {'id': 24922563, 'time': 62093.0}, {'id': 24922564, 'time': 63089.0}, {'id': -9713870, 'time': 64200.0}]}
{'id': 3237951, 'jobs': [{'id': -9713853, 'time': 36600.0}, {'id': 25037359, 'time': 42540.0}, {'id': 25037360, 'time': 44409.0}, {'id': 23926039, 'time

# Objectif2

In [28]:
# -- Ajout de la fonction objectif --
Z=0
for i in Pickups:
    C = {i : m.addConstr(quicksum( [Xijk[i,j,k] for k in chauffeurs for j in stations]) == 1, name = f'Constr{i}')}

for i in stations :
    for j in stations :
        for k in chauffeurs:
            Z+=Xijk[(i,j,k)]*time[(i,j)]
                
m.setObjective(Z, GRB.MINIMIZE)

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update()

#-- Affichage en mode texte du PL --
m.display()

In [29]:
start_time = tm.time()
# -- Résolution --
m.optimize()

In [30]:
from operator import itemgetter
if m.status == GRB.INFEASIBLE:
    print("\n LE PROGRAMME N'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print("\n LE PROGRAMME EST NON BORNÉ!!!")
else:
    client={}
    L=[]
    res=0
    attrib={}
    for k in chauffeurs:
        chauffeur=[]
        for i in stations:
            for j in stations:
                if Xijk[(i,j,k)].x==1 :
                    chauffeur.append((i,j,Bik[(i,k)].x))
                    res += time[(i,j)]
            attrib[k]=chauffeur
        c=0
        for i in Pickups:
            for j in stations:
                if Xijk[(i,j,k)].x==1 :
                    c+=1
        client[k]=c
        print("Le chauffeur {} effectue les courses suivantes: {}".format(k,chauffeur), "Il a {} clients".format(client[k]))
        L.append(sorted(chauffeur, key = itemgetter(2)))
print("Nombre maximal des clients servis :{}".format(sum(client[k] for k in chauffeurs)))
print("Temps total de trajet :{}".format(res)) 
print('Le temps d\'exécution de cette deuxième instance est: {} secondes'.format(tm.time() - start_time))

Le chauffeur 1 effectue les courses suivantes: [(0, 19, 45000.0), (12, 37, 46800.0), (15, 43, 49998.0), (18, 15, 48802.0), (19, 12, 46357.0), (37, 44, 47394.0), (40, 51, 51645.0), (43, 40, 50902.0), (44, 18, 47859.0)] Il a 4 clients
Le chauffeur 2 effectue les courses suivantes: [(0, 51, 31836.0)] Il a 0 clients
Le chauffeur 3 effectue les courses suivantes: [(0, 10, 52708.0), (10, 17, 55388.0), (14, 35, 56755.0), (17, 14, 56400.0), (24, 49, 61248.0), (35, 42, 57368.0), (39, 24, 58339.0), (42, 39, 57922.0), (49, 51, 63888.0)] Il a 4 clients
Le chauffeur 4 effectue les courses suivantes: [(0, 7, 36600.0), (1, 32, 42734.0), (3, 26, 44158.0), (7, 25, 42024.0), (20, 28, 44821.0), (25, 1, 42674.0), (26, 20, 44366.0), (28, 45, 45346.0), (32, 50, 43030.0), (45, 51, 45406.0), (50, 3, 44098.0)] Il a 5 clients
Le chauffeur 5 effectue les courses suivantes: [(0, 16, 48600.0), (5, 30, 57300.0), (8, 33, 50119.0), (11, 36, 53700.0), (16, 8, 49702.0), (30, 51, 58061.0), (33, 41, 51011.0), (36, 5, 545

In [32]:
chemin={}
for k in range(len(L)):
    chemin[k+1] = []
    for i in range(len(L[k])):
        chemin[k+1].append(L[k][i][0])
    chemin[k+1].append(51)
chemin

{1: [0, 19, 12, 37, 44, 18, 15, 43, 40, 51],
 2: [0, 51],
 3: [0, 10, 17, 14, 35, 42, 39, 24, 49, 51],
 4: [0, 7, 25, 1, 32, 50, 3, 26, 20, 28, 45, 51],
 5: [0, 16, 8, 33, 41, 11, 36, 5, 30, 51],
 6: [0, 51],
 7: [0, 2, 27, 4, 13, 38, 29, 9, 34, 51],
 8: [0, 22, 47, 23, 21, 46, 6, 31, 48, 51]}